In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

# 크롬 드라이버 실행
driver = uc.Chrome()
driver.get("https://play.google.com/store/apps/details?id=com.google.android.youtube&hl=ko")

# 리뷰 창 열기
driver.find_element(By.CSS_SELECTOR, 'c-wiz[jsrenderer="C7s1K"]').find_element(
    By.CSS_SELECTOR, "i.google-material-icons.notranslate.VfPpkd-kBDsod.W7A5Qb"
).click()

In [11]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

# 스크롤 가능한 div 찾기
scrollable_div = driver.find_element(By.CSS_SELECTOR, 'div.fysCi')

# 수집용 리스트
one_star_reviews = []
high_star_reviews = []
review_text_set = set()

while len(one_star_reviews) < 100 or len(high_star_reviews) < 100:
    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.RHo1pe')

    for review in reviews:
        try:
            rating_element = review.find_element(By.CSS_SELECTOR, 'div[role="img"]')
            rating_text = rating_element.get_attribute("aria-label")

            rating_match = re.search(r"(\d+)개 만점에 (\d+)개", rating_text)
            if rating_match:
                stars_received = int(rating_match.group(2))

                # 1점 또는 4/5점만 수집
                if (stars_received == 1 and len(one_star_reviews) < 100) or \
                   (stars_received in [4, 5] and len(high_star_reviews) < 100):

                    review_text = review.text.strip()
                    if review_text in review_text_set:
                        continue  # 중복 제거

                    review_text_set.add(review_text)

                    # 이름 추출
                    try:
                        name_element = review.find_element(By.CSS_SELECTOR, 'span.X5PpBb')
                        name = name_element.text.strip()
                    except:
                        name = "Unknown"

                    # 텍스트 정제
                    cleaned_review_text = review_text.replace('more_vert', '').replace(name, '').strip()

                    # 저장
                    row = [name, stars_received, cleaned_review_text]

                    if stars_received == 1:
                        one_star_reviews.append(row)
                    else:
                        high_star_reviews.append(row)

                    if len(one_star_reviews) >= 100 and len(high_star_reviews) >= 100:
                        break

        except Exception as e:
            print("❌ 오류 발생:", e)

    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
    time.sleep(2)

# 데이터프레임 생성 및 저장
collected_reviews = one_star_reviews + high_star_reviews
df = pd.DataFrame(collected_reviews, columns=["이름", "별점", "리뷰"])
df["리뷰"] = df["리뷰"].apply(lambda x: x.replace('\n', ' ').strip())

df.to_csv("유튜브 리뷰 크롤링_수정2.csv", index=False, encoding="utf-8-sig")
print("✅ 수집 완료! 1점:", len(one_star_reviews), "4,5점:", len(five_star_reviews))

✅ 수집 완료! 1점: 100 5점: 0


In [12]:
df

,이름,별점,리뷰
0,Unknown,1,김서태 2025년 4월 5일 아니 기초적인 테스트도 안하고 업데이트하고 배포하십니...
1,Unknown,1,시즈니로하블루 2025년 3월 28일 프리미엄 쓰고 있는 사람인데... 오프라인 ...
2,Unknown,1,가을유서 2025년 4월 6일 언젠가부터 업데이트 후 백그라운드 재생이 안되요. ...
3,Unknown,1,윤정호 2024년 11월 29일 아니 왜 화면 캡쳐할때 종종 엉뚱한 화면이 캡쳐되...
4,Unknown,1,황서희 (리안) 2024년 11월 2일 가끔 영상 화면이 까매져서 업데이트 안 한...
...,...,...,...
195,Unknown,5,"VVlP 2025년 2월 22일 다들 프리미엄 구독하실거면, 각오하세요. 무료 체..."
196,Unknown,5,가즈아 2025년 4월 10일 헤헤 완전 좋아요~ 이제 광고 두 개여도 상관 없어...
197,Unknown,4,ᅳ ᄉ ᅳ 2024년 10월 23일 솔직히 이번 업데이트 너무 불편해요. 영상 밑...
198,Unknown,4,최한나 2025년 4월 1일 광고 뜰때마다 소리가 안들리는 현상이 일어나는데 뭐가...


In [15]:
import re

def clean_review(text):
    # "이름 YYYY년 MM월 DD일" 제거
    return re.sub(r'^.*?\d{4}년 \d{1,2}월 \d{1,2}일\s*', '', text)

df['리뷰'] = df['리뷰'].apply(clean_review)
df

,이름,별점,리뷰
0,Unknown,1,아니 기초적인 테스트도 안하고 업데이트하고 배포하십니까? 누가 유튜브를 실행하자마나...
1,Unknown,1,프리미엄 쓰고 있는 사람인데... 오프라인 저장 관리가 어렵습니다.. 고쳐주세요ㅜㅜ...
2,Unknown,1,언젠가부터 업데이트 후 백그라운드 재생이 안되요. 화면이 꺼진 상태에서의 소리만 듣...
3,Unknown,1,아니 왜 화면 캡쳐할때 종종 엉뚱한 화면이 캡쳐되나요? 예를들어 제가 30초 화면 ...
4,Unknown,1,가끔 영상 화면이 까매져서 업데이트 안 한 탓인가 싶어서 업뎃 해도 까만 화면은 여...
...,...,...,...
195,Unknown,5,"다들 프리미엄 구독하실거면, 각오하세요. 무료 체험하고 못벗어납니다. 매달 구독할 ..."
196,Unknown,5,헤헤 완전 좋아요~ 이제 광고 두 개여도 상관 없어요~ 이젠 프리미엄 쓰니까 다 괜...
197,Unknown,4,솔직히 이번 업데이트 너무 불편해요. 영상 밑으로 내리면 다시 올려서 볼수가 없고 ...
198,Unknown,4,광고 뜰때마다 소리가 안들리는 현상이 일어나는데 뭐가문젤까요


In [16]:
df.to_csv("유튜브 리뷰 크롤링_수정.csv", index=False, encoding="utf-8-sig")

# 드라이버 종료
driver.quit()